### Imports

In [1]:
import sys
import pandas as pd
import numpy as np
import json
import re

### Pfade & Encoding

In [2]:
data_path = "./Hauptordner"
encoding = "utf-8"

### Zufallsseed für Reproduzierbarkeit

In [3]:
RANDOM_SEED = 42

### Dataset 3: champion_info2 – Grundchecks
- **Form & Größe**
- **Spaltennamen** und **Datentypen**
- **Erste Zeilen**
- **Zufällige Stichprobe**
- **Statistische Übersicht**
- **Fehlende Werte**
- **Duplikate**
- **Eindeutigkeit von Schlüsseln**
- **Min-/Max-Werte**
- **Wertebereiche**
- **Kardinalität**
- **Kategorische Verteilungen**
- **Auffälligkeiten markieren**
- **Konsistenz zwischen Spalten**
- **Komplett leere oder fast leere Spalten**
- **Datums-/Zeitspalten prüfen**
- **Zeichenketten in kategorischen Spalten prüfen**
- **Patch-Version prüfen**
- **Sonderzeichen in Namen prüfen**
- **Key vs. Name Schreibweise prüfen**
- **Titel auf Plausibilität prüfen**
- **Namens-Eindeutigkeit**
- **Prüfung auf Platzhalter-Champions**
- **Tags-Spalte prüfen**
- **Anzahl Tags pro Champion prüfen**
- **Ausreißer Checks**

In [4]:
df_champ2 = pd.read_json("champion_info_2.json")
print(df_champ2.columns)
df_champ2_raw = df_champ2.copy()
df_champ2 = df_champ2["data"].apply(pd.Series)

# Champ2 ist Verschachtelt deswegen muss ich dieses vorher ausführen.Für spätere checks in Champ1.

Index(['type', 'version', 'data'], dtype='object')


In [5]:
print(df_champ2.columns)

Index(['tags', 'title', 'id', 'key', 'name'], dtype='object')


### Form & Größe

In [6]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info_2.json"
df_champ2 = pd.read_json(pfad)
df_champ2.shape

(139, 3)

### Spaltennamen und Datentypen

In [7]:
df_champ2.dtypes

type       object
version    object
data       object
dtype: object

### Erstenzeilen anzeigen

In [8]:
df_champ2.head()

,type,version,data
None,champion,7.18.1,"{'tags': [], 'title': 'None', 'id': -1, 'key':..."
MonkeyKing,champion,7.18.1,"{'tags': ['Fighter', 'Tank'], 'title': 'the Mo..."
Jax,champion,7.18.1,"{'tags': ['Fighter', 'Assassin'], 'title': 'Gr..."
Fiddlesticks,champion,7.18.1,"{'tags': ['Mage', 'Support'], 'title': 'the Ha..."
Shaco,champion,7.18.1,"{'tags': ['Assassin'], 'title': 'the Demon Jes..."


### Zufällige Stichprobe

In [9]:
df_champ2.sample(5, random_state=1)

,type,version,data
Shen,champion,7.18.1,"{'tags': ['Tank'], 'title': 'the Eye of Twilig..."
Annie,champion,7.18.1,"{'tags': ['Mage'], 'title': 'the Dark Child', ..."
Irelia,champion,7.18.1,"{'tags': ['Fighter', 'Assassin'], 'title': 'th..."
Mordekaiser,champion,7.18.1,"{'tags': ['Fighter'], 'title': 'the Iron Reven..."
Warwick,champion,7.18.1,"{'tags': ['Fighter', 'Tank'], 'title': 'the Un..."


### Statische Übersicht

In [10]:
df_champ2.describe

<bound method NDFrame.describe of                   type version  \
None          champion  7.18.1   
MonkeyKing    champion  7.18.1   
Jax           champion  7.18.1   
Fiddlesticks  champion  7.18.1   
Shaco         champion  7.18.1   
...                ...     ...   
KogMaw        champion  7.18.1   
Ahri          champion  7.18.1   
Quinn         champion  7.18.1   
Leblanc       champion  7.18.1   
Ezreal        champion  7.18.1   

                                                           data  
None          {'tags': [], 'title': 'None', 'id': -1, 'key':...  
MonkeyKing    {'tags': ['Fighter', 'Tank'], 'title': 'the Mo...  
Jax           {'tags': ['Fighter', 'Assassin'], 'title': 'Gr...  
Fiddlesticks  {'tags': ['Mage', 'Support'], 'title': 'the Ha...  
Shaco         {'tags': ['Assassin'], 'title': 'the Demon Jes...  
...                                                         ...  
KogMaw        {'tags': ['Marksman', 'Mage'], 'title': 'the M...  
Ahri          {'tags': ['Mage

### Fehlende Werte

In [11]:
df_champ2.isnull().sum()

type       0
version    0
data       0
dtype: int64

### Duplikate überprüfen

In [12]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info_2.json"
data = pd.read_json(pfad)
champ_dict = data.loc[data.index[0], 'data']
df_champ2 = pd.json_normalize(champ_dict.values())

df_champ2.duplicated().sum()

np.int64(0)

### Eindeutigkeit von Schlüssel

In [13]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info_2.json"
data = pd.read_json(pfad)
champ_dict = data['data'].to_dict()
df_champ2 = pd.json_normalize(champ_dict.values())

df_champ2['key'].is_unique

True

### MIN-/MAX-Werte je Spalte

In [14]:
df_champ2.describe().loc[['min', 'max']]

,id
min,-1.0
max,516.0


### Wertebereiche

In [15]:
df_champ2.describe()

,id
count,139.000000
mean,107.877698
std,111.348614
min,-1.000000
25%,34.500000
50%,78.000000
75%,126.500000
max,516.000000


### Kardinalität der Spalten

In [16]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info_2.json"
data = pd.read_json(pfad)
champ_dict = data['data'][0]
df_champ2 = pd.json_normalize(champ_dict.values())

df_champ2.nunique()

C:\Users\aljos\AppData\Local\Temp\ipykernel_8080\3322033045.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  champ_dict = data['data'][0]


Series([], dtype: float64)

### Kategorische Verteilungen

In [17]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info_2.json"
with open(pfad, encoding='utf-8') as f:
    raw = json.load(f)
champ_dict = raw['data']
champ_liste = list(champ_dict.values())
df_champ2 = pd.json_normalize(champ_liste)
df_champ2['tags'].explode().value_counts()

tags
Fighter     68
Mage        52
Tank        41
Assassin    34
Support     28
Marksman    25
Name: count, dtype: int64

### Auffälligkeiten markieren

In [18]:
df_champ2['auffällig_leer'] = df_champ2.isnull().any(axis=1)
df_champ2['auffällig_doppelt'] = df_champ2.duplicated(subset='id', keep=False)
df_champ2['auffällig_titel'] = df_champ2['title'].str.len() > 50
df_champ2[df_champ2[['auffällig_leer', 'auffällig_doppelt', 'auffällig_titel']].any(axis=1)]

,tags,title,id,key,name,auffällig_leer,auffällig_doppelt,auffällig_titel


### Konsistenz zwischen Spalten

In [19]:
df_champ2['konsistenz_id_key'] = df_champ2['id'].astype(str) == df_champ2['key'].astype(str)
df_champ2[~df_champ2['konsistenz_id_key']]

,tags,title,id,key,name,auffällig_leer,auffällig_doppelt,auffällig_titel,konsistenz_id_key
0,[],None,-1,None,None,False,False,False,False
1,"[Fighter, Tank]",the Monkey King,62,MonkeyKing,Wukong,False,False,False,False
2,"[Fighter, Assassin]",Grandmaster at Arms,24,Jax,Jax,False,False,False,False
3,"[Mage, Support]",the Harbinger of Doom,9,Fiddlesticks,Fiddlesticks,False,False,False,False
4,[Assassin],the Demon Jester,35,Shaco,Shaco,False,False,False,False
...,...,...,...,...,...,...,...,...,...
134,"[Marksman, Mage]",the Mouth of the Abyss,96,KogMaw,Kog'Maw,False,False,False,False
135,"[Mage, Assassin]",the Nine-Tailed Fox,103,Ahri,Ahri,False,False,False,False
136,"[Marksman, Fighter]",Demacia's Wings,133,Quinn,Quinn,False,False,False,False
137,"[Assassin, Mage]",the Deceiver,7,Leblanc,LeBlanc,False,False,False,False


### Komplett leere oder fast leere Spalten

In [20]:
leere_spalten = df_champ2.columns[df_champ2.isnull().all()]
fast_leere_spalten = df_champ2.columns[df_champ2.isnull().mean() > 0.9]

print("Komplett leere Spalten:", list(leere_spalten))
print("Fast leere Spalten (>90% NaN):", list(fast_leere_spalten))

Komplett leere Spalten: []
Fast leere Spalten (>90% NaN): []


### Datums-/Zeitspalten prüfen

In [21]:
kategorie_spalten = df_champ2.select_dtypes(include=['object']).columns
print("Gefundene Textspalten:", list(kategorie_spalten))

Gefundene Textspalten: ['tags', 'title', 'key', 'name']


In [22]:
kategorie_spalten = df_champ2.select_dtypes(include=['object']).columns

if len(kategorie_spalten) == 0:
    print("Keine kategorischen (Text-)Spalten gefunden.")
else:
    for spalte in kategorie_spalten:
        print(f"Spalte: {spalte}")
        print(df_champ2[spalte].str.len().describe())
        print("-" * 40)

Spalte: tags
count    139.000000
mean       1.784173
std        0.430076
min        0.000000
25%        2.000000
50%        2.000000
75%        2.000000
max        2.000000
Name: tags, dtype: float64
----------------------------------------
Spalte: title
count    139.000000
mean      17.812950
std        3.982877
min        4.000000
25%       15.000000
50%       18.000000
75%       21.000000
max       27.000000
Name: title, dtype: float64
----------------------------------------
Spalte: key
count    139.000000
mean       6.057554
std        1.929067
min        2.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       12.000000
Name: key, dtype: float64
----------------------------------------
Spalte: name
count    139.000000
mean       6.136691
std        2.022344
min        2.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       12.000000
Name: name, dtype: float64
----------------------------------------


### Zeichenketten in kategorischen Spalten

In [23]:
kategorie_spalten = df_champ2.select_dtypes(include=['object']).columns
print("Gefundene Textspalten:", list(kategorie_spalten))

Gefundene Textspalten: ['tags', 'title', 'key', 'name']


In [24]:
kategorie_spalten = df_champ2.select_dtypes(include=['object']).columns
if len(kategorie_spalten) == 0:
    print("Keine kategorischen (Text-)Spalten gefunden.")
else:
    for spalte in kategorie_spalten:
        print(f"Spalte: {spalte}")
        print(df_champ2[spalte].str.len().describe())
        print("-" * 40)

Spalte: tags
count    139.000000
mean       1.784173
std        0.430076
min        0.000000
25%        2.000000
50%        2.000000
75%        2.000000
max        2.000000
Name: tags, dtype: float64
----------------------------------------
Spalte: title
count    139.000000
mean      17.812950
std        3.982877
min        4.000000
25%       15.000000
50%       18.000000
75%       21.000000
max       27.000000
Name: title, dtype: float64
----------------------------------------
Spalte: key
count    139.000000
mean       6.057554
std        1.929067
min        2.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       12.000000
Name: key, dtype: float64
----------------------------------------
Spalte: name
count    139.000000
mean       6.136691
std        2.022344
min        2.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       12.000000
Name: name, dtype: float64
----------------------------------------


### Patch-Version

In [25]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info_2.json"
with open(pfad, encoding='utf-8') as f:
    roh = json.load(f)
champ_dict = roh['data']
champ_liste = list(champ_dict.values())
df_champ2 = pd.json_normalize(champ_liste)
df_champ2['version'] = roh['version']

print("Verschiedene Patch-Versionen:")
print(df_champ2['version'].value_counts())

Verschiedene Patch-Versionen:
version
7.18.1    139
Name: count, dtype: int64


### Sonderzeichen in Namen

In [26]:
sonderzeichen = df_champ2['name'].apply(lambda x: re.findall(r'[^a-zA-Z0-9\s]', x))
df_champ2[sonderzeichen.apply(lambda x: len(x) > 0)]

,tags,title,id,key,name,version
52,"[Assassin, Fighter]",the Voidreaver,121,Khazix,Kha'Zix,7.18.1
56,"[Fighter, Tank]",the Madman of Zaun,36,DrMundo,Dr. Mundo,7.18.1
75,"[Tank, Mage]",the Terror of the Void,31,Chogath,Cho'Gath,7.18.1
94,[Mage],the Eye of the Void,161,Velkoz,Vel'Koz,7.18.1
121,[Fighter],the Void Burrower,421,RekSai,Rek'Sai,7.18.1
134,"[Marksman, Mage]",the Mouth of the Abyss,96,KogMaw,Kog'Maw,7.18.1


### Key vs. Name Schreibweise

In [27]:
df_champ2['name_lower'] = df_champ2['name'].str.lower()
df_champ2['key_lower'] = df_champ2['key'].str.lower()
df_champ2[df_champ2['name_lower'] != df_champ2['key_lower']]

,tags,title,id,key,name,version,name_lower,key_lower
1,"[Fighter, Tank]",the Monkey King,62,MonkeyKing,Wukong,7.18.1,wukong,monkeyking
17,[Marksman],the Bounty Hunter,21,MissFortune,Miss Fortune,7.18.1,miss fortune,missfortune
52,"[Assassin, Fighter]",the Voidreaver,121,Khazix,Kha'Zix,7.18.1,kha'zix,khazix
56,"[Fighter, Tank]",the Madman of Zaun,36,DrMundo,Dr. Mundo,7.18.1,dr. mundo,drmundo
74,[Mage],the Card Master,4,TwistedFate,Twisted Fate,7.18.1,twisted fate,twistedfate
75,"[Tank, Mage]",the Terror of the Void,31,Chogath,Cho'Gath,7.18.1,cho'gath,chogath
94,[Mage],the Eye of the Void,161,Velkoz,Vel'Koz,7.18.1,vel'koz,velkoz
101,"[Assassin, Fighter]",the Wuju Bladesman,11,MasterYi,Master Yi,7.18.1,master yi,masteryi
111,"[Support, Tank]",the River King,223,TahmKench,Tahm Kench,7.18.1,tahm kench,tahmkench
113,"[Fighter, Assassin]",the Seneschal of Demacia,5,XinZhao,Xin Zhao,7.18.1,xin zhao,xinzhao


### Titel auf Plausibilität

In [28]:
if 'title' in df_champ2.columns:
    print("Übersicht der Titel:")
    print(df_champ2['title'].value_counts())
else:
    print("Spalte 'title' nicht vorhanden.")

Übersicht der Titel:
title
None                      1
the Monkey King           1
Grandmaster at Arms       1
the Harbinger of Doom     1
the Demon Jester          1
                         ..
the Mouth of the Abyss    1
the Nine-Tailed Fox       1
Demacia's Wings           1
the Deceiver              1
the Prodigal Explorer     1
Name: count, Length: 139, dtype: int64


### Namens-Eindeutigkeit

In [29]:
doppelte_namen = df_champ2['name'].duplicated().sum()
print(f"Anzahl doppelter Namen: {doppelte_namen}")

Anzahl doppelter Namen: 0


### Platzhalter-Champions

In [30]:
df_champ2[df_champ2['id'] == -1]

,tags,title,id,key,name,version,name_lower,key_lower
0,[],None,-1,None,None,7.18.1,none,none


### Tags-Spalte

In [31]:
df_champ2.explode('tags')['tags'].value_counts()

tags
Fighter     68
Mage        52
Tank        41
Assassin    34
Support     28
Marksman    25
Name: count, dtype: int64

### Anzahl Tags pro Champion

In [32]:
df_champ2['anzahl_tags'] = df_champ2['tags'].apply(len)
print("Anzahl Tags je Champion:")
print(df_champ2['anzahl_tags'].value_counts().sort_index())

Anzahl Tags je Champion:
anzahl_tags
0      1
1     28
2    110
Name: count, dtype: int64


### Ausreißer Checks

In [33]:
# Auch hier sind die numerischen Spalten (id, key, anzahl_tags) nur IDs oder Kategorien. 
# Die Werte sind fest definiert und plausibel, somit keine echten Ausreißer

### Dataset 3: champion_info2 – Spezifische Checks
- **Sonderzeichen im Namen prüfen**
- **Key-Format-Prüfung (id, key, name)**
- **Leere Strings prüfen**
- **Doppelte id oder key prüfen**
- **Gleicher name für mehrere ids prüfen**
- **Negativwerte in id prüfen (z. B. -1)**
- **Spalten name_lower und key_lower logisch korrekt**
- **Titel auf Plausibilität prüfen**

### Sonderzeichen im Namen

In [34]:
pattern = re.compile(r"^[A-Za-zÀ-ÿ .'-]+$")

sonderzeichen = df_champ2[~df_champ2["name"].apply(lambda x: bool(pattern.match(x)))]
print("Champions mit Sonderzeichen im Namen:")
print(sonderzeichen[["id", "name"]])

Champions mit Sonderzeichen im Namen:
Empty DataFrame
Columns: [id, name]
Index: []


### Key-Format-Prüfung

In [35]:
print("Datentypen prüfen:")
print(df_champ2[["id", "key", "name"]].dtypes)

print("\nUngültige IDs (nicht ganzzahlig):")
print(df_champ2[~df_champ2["id"].apply(lambda x: isinstance(x, int))])

Datentypen prüfen:
id       int64
key     object
name    object
dtype: object

Ungültige IDs (nicht ganzzahlig):
Empty DataFrame
Columns: [tags, title, id, key, name, version, name_lower, key_lower, anzahl_tags]
Index: []


### Leere Strings

In [36]:
spalten = ["id", "key", "name"]
for spalte in spalten:
    leere = df_champ2[df_champ2[spalte].astype(str).str.strip() == ""]
    print(f"Leere Einträge in '{spalte}': {len(leere)}")
    if not leere.empty:
        print(leere[spalte])

Leere Einträge in 'id': 0
Leere Einträge in 'key': 0
Leere Einträge in 'name': 0


### Doppelte id oder key

In [37]:
doppelte_ids = df_champ2[df_champ2["id"].duplicated()]
print("Doppelte IDs:")
print(doppelte_ids[["id", "name"]])

doppelte_keys = df_champ2[df_champ2["key"].duplicated()]
print("\nDoppelte Keys:")
print(doppelte_keys[["key", "name"]])

Doppelte IDs:
Empty DataFrame
Columns: [id, name]
Index: []

Doppelte Keys:
Empty DataFrame
Columns: [key, name]
Index: []


### Gleicher name für mehrere ids

In [38]:
mehrfach_namen = df_champ2.groupby("name")["id"].nunique()
doppelte_namen = mehrfach_namen[mehrfach_namen > 1]

print("Gleicher Name mit mehreren IDs:")
print(doppelte_namen)

Gleicher Name mit mehreren IDs:
Series([], Name: id, dtype: int64)


### Negativwerte in id

In [39]:
negative_ids = df_champ2[df_champ2["id"] < 0]
print("Negative IDs:")
print(negative_ids[["id", "name"]])

# -1 steht für kein Ban ist im platzhalter gespeichert.

Negative IDs:
   id  name
0  -1  None


In [40]:
df_champ2[df_champ2["id"] == -1]

,tags,title,id,key,name,version,name_lower,key_lower,anzahl_tags
0,[],None,-1,None,None,7.18.1,none,none,0


In [41]:
df_champ2[df_champ2["id"] == -1][["id", "name", "title", "key"]]

,id,name,title,key
0,-1,None,None,None


In [42]:
df_champ2.loc[df_champ2["id"] == -1, "id"] = 0
df_champ2.loc[df_champ2["id"] == 0, ["name", "title", "key", "name_lower", "key_lower"]] = "Placeholder"

# -1 in 0 Umgewandelt.

In [43]:
placeholder_check = df_champ2[df_champ2["id"] == 0]
print("Platzhalter-Datensatz (id = 0):")
print(placeholder_check[["id", "name", "title", "key"]])

# Gegencheck

Platzhalter-Datensatz (id = 0):
   id         name        title          key
0   0  Placeholder  Placeholder  Placeholder


### Spalten name_lower und key_lower logisch korrekt

In [44]:
name_check = df_champ2["name_lower"] != df_champ2["name"].str.lower()
key_check = df_champ2["key_lower"] != df_champ2["key"].str.lower()

fehlerhafte_namen = df_champ2[name_check][["name", "name_lower"]]
fehlerhafte_keys = df_champ2[key_check][["key", "key_lower"]]

print("Abweichungen bei name_lower:")
print(fehlerhafte_namen)

print("\nAbweichungen bei key_lower:")
print(fehlerhafte_keys)

Abweichungen bei name_lower:
          name   name_lower
0  Placeholder  Placeholder

Abweichungen bei key_lower:
           key    key_lower
0  Placeholder  Placeholder


### Titel auf Plausibilität

In [45]:
ungueltige_title = df_champ2["title"].isna() | (df_champ2["title"] == "") | (df_champ2["title"].str.lower() == "placeholder")
df_champ2[ungueltige_title][["id", "name", "title"]]

,id,name,title
0,0,Placeholder,Placeholder


### Cross-Dataset-Checks
- **Champion-IDs aus games in champ2 vorhanden?**
- **Gibt es IDs in champ2, die nie in games vorkommen?**
- **Wurde -1 in games durch 0 ersetzt?**
- **Name-Schreibweise konsistent zwischen champ1 und champ2**
- **id-Vergleich mit champ1: Gibt es Unterschiede?**
- **Doppelte name oder key in Kombination mit champ1**
- **Verlinkung id  name oder key für spätere Merges korrekt?**

### Champion-IDs aus games in champ2 vorhanden?

In [46]:
df = pd.read_csv("games.csv")
champ2_ids = set(df_champ2["id"])
champ_columns = [
    "t1_champ1id", "t1_champ2id", "t1_champ3id", "t1_champ4id", "t1_champ5id",
    "t2_champ1id", "t2_champ2id", "t2_champ3id", "t2_champ4id", "t2_champ5id"
]
game_ids = set(df[champ_columns].values.ravel())
fehlende_ids = game_ids - champ2_ids

print("Champion-IDs in Games, aber nicht in Champion 2:")
print(fehlende_ids)

Champion-IDs in Games, aber nicht in Champion 2:
set()


### Gibt es IDs in champ2, die nie in games vorkommen?

In [47]:
champ2_ids = set(df_champ2["id"])
champ_columns = [
    "t1_champ1id", "t1_champ2id", "t1_champ3id", "t1_champ4id", "t1_champ5id",
    "t2_champ1id", "t2_champ2id", "t2_champ3id", "t2_champ4id", "t2_champ5id"
]
game_ids = set(df[champ_columns].values.ravel())
unbenutzte_ids = champ2_ids - game_ids

print("Champion-IDs in Champion 2, aber nicht in Games:")
print(unbenutzte_ids)

Champion-IDs in Champion 2, aber nicht in Games:
{0}


### Wurde -1 in games durch 0 ersetzt?

In [48]:
games2 = pd.read_csv("games2.csv")
for spalte in games2.columns:
    anzahl = (games2[spalte] == -1).sum()
    if anzahl > 0:
        print(f"{spalte}: {anzahl}x -1")

### Name-Schreibweise konsistent zwischen champ1 und champ2

In [49]:
df_champ1 = pd.read_csv("champion1.0.csv")

namen_champ1 = df_champ1[["id", "name"]].copy()
namen_champ2 = df_champ2[["id", "name"]].copy()

namen_merge = pd.merge(namen_champ1, namen_champ2, on="id", suffixes=("_champ1", "_champ2"))
unterschiede = namen_merge[namen_merge["name_champ1"] != namen_merge["name_champ2"]]
print("Anzahl unterschiedlicher Namen bei gleicher ID:", len(unterschiede))
display(unterschiede)

Anzahl unterschiedlicher Namen bei gleicher ID: 0


,id,name_champ1,name_champ2


### id-Vergleich mit champ1

In [50]:
ids_champ1 = set(df_champ1["id"])
ids_champ2 = set(df_champ2["id"])

nur_in_champ1 = ids_champ1 - ids_champ2
nur_in_champ2 = ids_champ2 - ids_champ1

print("IDs nur in Champion 1:", nur_in_champ1)
print("IDs nur in Champion 2:", nur_in_champ2)

IDs nur in Champion 1: set()
IDs nur in Champion 2: {0}


### Doppelte name oder key in Kombination mit champ1

In [51]:
kombis_champ1 = set(zip(df_champ1["name"], df_champ1["key"]))
kombis_champ2 = set(zip(df_champ2["name"], df_champ2["key"]))

gemeinsame_kombis = kombis_champ1 & kombis_champ2
nur_in_champ1 = kombis_champ1 - kombis_champ2
nur_in_champ2 = kombis_champ2 - kombis_champ1

print("Kombis nur in Champion 1:", nur_in_champ1)
print("Kombis nur in Champion 2:", nur_in_champ2)

Kombis nur in Champion 1: set()
Kombis nur in Champion 2: {('Placeholder', 'Placeholder')}


### Verlinkung id  name oder key

In [52]:
print("Champion 1:")
print("IDs:", df_champ1["id"].nunique())
print("Namen:", df_champ1["name"].nunique())
print("Keys:", df_champ1["key"].nunique())

print("\nChampion 2:")
print("IDs:", df_champ2["id"].nunique())
print("Namen:", df_champ2["name"].nunique())
print("Keys:", df_champ2["key"].nunique())

# Bei Champion 2 einer mehr wegen dem Platzhalter (-1 zu 0 umgewandelt)

Champion 1:
IDs: 138
Namen: 138
Keys: 138

Champion 2:
IDs: 139
Namen: 139
Keys: 139


In [53]:
ids_match = set(df_champ1["id"]) & set(df_champ2["id"])
print("Gemeinsame IDs:", len(ids_match))

namen_match = set(df_champ1["name"]) & set(df_champ2["name"])
print("Gemeinsame Namen:", len(namen_match))

keys_match = set(df_champ1["key"]) & set(df_champ2["key"])
print("Gemeinsame Keys:", len(keys_match))

Gemeinsame IDs: 138
Gemeinsame Namen: 138
Gemeinsame Keys: 138


### Abspeichern und eine Copi erstellen

In [54]:
df_champ2.to_csv("champion2.0.csv", index=False)
print("Sicherheitskopie erfolgreich gespeichert: 'champion2.0.csv'")

Sicherheitskopie erfolgreich gespeichert: 'champion2.0.csv'
